# Analysis Sub-Agent Test Notebook

This notebook tests the analysis sub-agent, which is responsible for:
- Data analysis and processing
- Creating visualizations (saved to deep-agent/scratchpad/plots)
- Statistical analysis and trend identification
- Supporting the main agent's Markets Research & Portfolio Risk Orchestration goals

The main agent uses this sub-agent to:
- Analyze equity and factor data
- Generate price reaction analysis
- Create correlation, beta, and sector aggregation visualizations
- Execute any task requiring code execution, charts, or numerical summaries

## Prerequisites

This notebook connects to the LangGraph server, which provides:
- **Cloud-hosted store** - Persistent memory across sessions (same as LangSmith Studio)
- **Cloud-hosted checkpointer** - Conversation state persistence

### Setup Steps

1. **Start the LangGraph server** (from the `deep-agent/` directory):
   ```bash
   cd deep-agent
   langgraph dev
   ```

2. **Wait for the server** to be ready at `http://localhost:2024`

3. **Run this notebook** - it connects to the same agent as LangSmith Studio

### Why This Architecture?

When you run `langgraph dev`, LangGraph connects to LangSmith's cloud infrastructure.
This means:
- Agent state persists across sessions
- No local PostgreSQL database required
- Same agent instance as LangSmith Studio

## Setup

In [ ]:
# Ensure scratchpad folders exist and are empty
from pathlib import Path
import shutil

scratchpad = Path("../scratchpad")
for folder in ["data", "images", "notes", "plots", "reports"]:
    path = scratchpad / folder
    if path.exists():
        shutil.rmtree(path)
    path.mkdir(parents=True)
    
print("Scratchpad folders ready (data, images, notes, plots, reports)")

In [ ]:

from langgraph_sdk import get_sync_client
from dotenv import load_dotenv

load_dotenv()

# Connect to local LangGraph server (must have `langgraph dev` running)
LANGGRAPH_URL = "http://localhost:2024"

try:
    client = get_sync_client(url=LANGGRAPH_URL)
    assistants = client.assistants.search(limit=10)
    if not assistants:
        raise RuntimeError("No assistants registered. Run `langgraph dev` from the deep-agent/ directory and try again.")

    ASSISTANT_ID = assistants[0]["assistant_id"]

    print(f"Connected to LangGraph server at {LANGGRAPH_URL}")
    print("Available assistants:")
    for a in assistants:
        marker = " (selected)" if a["assistant_id"] == ASSISTANT_ID else ""
        print(f"  - {a['assistant_id']}: {a.get('name', 'unnamed')}{marker}")
    print(f"Using assistant_id: {ASSISTANT_ID}")
except Exception as e:
    print(f"Could not connect to LangGraph server at {LANGGRAPH_URL}")
    print(f"Error: {e}")
    print("Make sure to run 'langgraph dev' from the deep-agent/ directory first.")
    raise SystemExit(1)


## Helper Function to Test the Agent

In [ ]:
import time
from IPython.display import display, Markdown


def truncate(text, limit=2000):
    return text[:limit] + "\n..." if len(text) > limit else text


def test_analysis_agent(message: str, thread_id: str = None):
    """Run the analysis agent via LangGraph SDK and display all intermediate steps."""
    thread_id = thread_id or f"test-{int(time.time())}"

    try:
        client.threads.get(thread_id)
    except Exception:
        client.threads.create(thread_id=thread_id)

    display(Markdown(f"## Task\n```\n{message.strip()}\n```\n---"))

    final_response = None

    for chunk in client.runs.stream(
        thread_id=thread_id,
        assistant_id=ASSISTANT_ID,
        input={"messages": [{"role": "user", "content": message}]},
        stream_mode="updates",
    ):
        if chunk.event == "updates":
            for node_name, node_output in chunk.data.items():
                messages = node_output.get("messages", [])
                for msg in messages:
                    msg_type = msg.get("type")

                    if msg_type == "ai" and msg.get("tool_calls"):
                        for tc in msg["tool_calls"]:
                            name = tc.get("name")
                            args = tc.get("args", {})
                            if name == "execute_python" and "code" in args:
                                display(Markdown(f"### Tool Call: `{name}`\n```python\n{truncate(args['code'], 1500)}\n```"))
                            else:
                                display(Markdown(f"### Tool Call: `{name}`\n```json\n{truncate(str(args), 500)}\n```"))

                    elif msg_type == "tool":
                        content = msg.get("content", "")
                        display(Markdown(f"### Tool Response\n```\n{truncate(content)}\n```\n---"))

                    elif msg_type == "ai" and msg.get("content") and not msg.get("tool_calls"):
                        final_response = msg["content"]
                        display(Markdown(f"## Response\n{final_response}"))

    return final_response


---
# Example 1: Basic Price Movement Visualization (Simple)

**Context**: A trader wants to quickly visualize recent price movements for a single stock.

**Sub-agent role**: Create a simple time-series visualization showing price trends.

In [ ]:
# Example 1: Simple price movement analysis
example_1_message = """I need you to create a simple price movement chart for analysis.

Here's the price data for TSLA over the last 5 trading days:

Date,Close,Volume
2025-12-15,385.50,125000000
2025-12-16,392.30,138000000
2025-12-17,388.75,115000000
2025-12-18,395.20,142000000
2025-12-19,401.85,156000000

Please create a clean line chart showing the closing prices over time. 
Save it to the outputs directory so I can include it in my report.
"""

import time
response_1 = test_analysis_agent(example_1_message, thread_id=f"example-1-{int(time.time())}")

---
# Example 2: Sector Correlation Analysis (Medium)

**Context**: A portfolio manager wants to understand how different tech stocks moved together during a recent market event.

**Sub-agent role**: Calculate correlations between multiple stocks and create a correlation heatmap to identify risk concentrations.

In [ ]:
# Example 2: Sector correlation analysis
example_2_message = """Analyze the correlation between major tech stocks during the last 10 trading days.

Here's the daily return data (%):

Date,AAPL,MSFT,GOOGL,META,NVDA
2025-12-09,0.5,0.3,0.8,1.2,2.1
2025-12-10,-0.8,-0.5,-1.1,-1.3,-2.5
2025-12-11,1.2,0.9,1.5,1.8,3.2
2025-12-12,-0.3,-0.2,-0.4,-0.6,-0.9
2025-12-13,0.9,0.7,1.1,1.4,2.3
2025-12-16,-1.5,-1.2,-1.8,-2.1,-3.4
2025-12-17,1.8,1.4,2.2,2.5,4.1
2025-12-18,0.4,0.3,0.5,0.7,1.1
2025-12-19,-0.6,-0.4,-0.8,-1.0,-1.6
2025-12-20,1.1,0.8,1.3,1.6,2.7

Please:
1. Calculate the correlation matrix between these stocks
2. Create a heatmap visualization showing the correlations
3. Identify which stocks are most correlated (potential concentration risk)
4. Save the visualization for inclusion in a risk report
"""

response_2 = test_analysis_agent(example_2_message, thread_id=f"example-2-{int(time.time())}")

---
# Example 3: Multi-Asset Event Impact Analysis (Complex)

**Context**: After a major Fed announcement, a risk manager needs to understand the cross-asset impact on their portfolio, including equities, bonds, and commodities.

**Sub-agent role**: Perform comprehensive analysis including:
- Price reaction analysis across multiple asset classes
- Volatility spike detection
- Statistical significance testing
- Multiple coordinated visualizations
- Portfolio-level impact assessment

In [ ]:
# Example 3: Complex multi-asset event impact analysis
# First, write the data to CSV files in scratchpad/data

import pandas as pd
from pathlib import Path
from IPython.display import display, Markdown

display(Markdown("## Data Setup\n*Writing CSV files to scratchpad/data...*"))

# Create the data directory
data_dir = Path("../scratchpad/data")
data_dir.mkdir(parents=True, exist_ok=True)

# Equity indices data
equity_data = """Time,SPY,QQQ,IWM
13:00,0.0,0.0,0.0
13:30,0.1,0.2,0.0
14:00,0.2,0.3,0.1
14:30,1.5,2.1,1.2
15:00,1.8,2.5,1.4
15:30,1.6,2.3,1.3
16:00,1.5,2.2,1.2"""

# Bond yields data
bonds_data = """Time,UST_2Y,UST_10Y,UST_30Y
13:00,0,0,0
13:30,1,0,0
14:00,2,1,1
14:30,-8,-12,-10
15:00,-10,-15,-12
15:30,-9,-14,-11
16:00,-8,-13,-11"""

# Commodities data
commodities_data = """Time,Gold,Oil,Dollar_Index
13:00,0.0,0.0,0.0
13:30,0.1,-0.1,0.0
14:00,0.2,-0.1,0.1
14:30,1.8,-1.5,-1.2
15:00,2.1,-1.8,-1.4
15:30,2.0,-1.7,-1.3
16:00,1.9,-1.6,-1.2"""

# Write CSV files
import io
pd.read_csv(io.StringIO(equity_data)).to_csv(data_dir / "fed_event_equities.csv", index=False)
pd.read_csv(io.StringIO(bonds_data)).to_csv(data_dir / "fed_event_bonds.csv", index=False)
pd.read_csv(io.StringIO(commodities_data)).to_csv(data_dir / "fed_event_commodities.csv", index=False)

print(f"Data files written to {data_dir.resolve()}")
for f in data_dir.glob("*.csv"):
    print(f"   - {f.name}")

In [ ]:
# Now run the analysis agent
from IPython.display import display, Markdown

display(Markdown("---\n## AI Agent Task\n*Sending task to analysis agent...*"))

example_3_message = """Analyze the market impact of the Fed rate decision announced on 2025-12-18 at 2:00 PM EST.

I need a comprehensive analysis across multiple asset classes. The data is stored in CSV files:

- Equity indices (Intraday % change, 30-min intervals): scratchpad/data/fed_event_equities.csv
- Bond yields (Basis points change): scratchpad/data/fed_event_bonds.csv  
- Commodities (% change): scratchpad/data/fed_event_commodities.csv

PORTFOLIO EXPOSURES (as % of total portfolio):
SPY: 35%
QQQ: 25%
IWM: 10%
UST_10Y: 20%
Gold: 5%
Oil: 5%

Please provide:
1. Multi-panel visualization showing price reactions across all asset classes with a vertical line at 14:00 (announcement time)
2. Calculate the portfolio-level impact based on the exposures provided
3. Identify which asset showed the most significant reaction (using statistical measures)
4. Calculate the realized volatility spike (comparing 30 min before vs 30 min after the announcement)
5. Create a summary table showing:
   - Asset
   - Max intraday move
   - Impact on portfolio (%)
   - Statistical significance (t-stat comparing pre/post volatility)

Save all visualizations with descriptive names. This will go into a risk committee presentation.
"""

response_3 = test_analysis_agent(example_3_message, thread_id=f"example-3-{int(time.time())}")

---
# Verify Output Files

Check what plots were created in the scratchpad/plots directory.

In [ ]:
import os
from pathlib import Path

plots_dir = Path("../scratchpad/plots")

if plots_dir.exists():
    plot_files = list(plots_dir.glob("*.png"))
    print(f"Found {len(plot_files)} plots in {plots_dir}:\n")
    for plot in sorted(plot_files, key=lambda x: x.stat().st_mtime, reverse=True):
        print(f"  - {plot.name}")
else:
    print(f"Directory {plots_dir} does not exist yet")

---
# Notes

## Expected Outputs

For each example, the analysis agent should:
1. **Process the data** provided in the message
2. **Execute Python code** to perform the requested analysis
3. **Generate visualizations** saved to `/home/daytona/outputs/` (auto-downloaded to `scratchpad/plots/`)
4. **Return a response** containing:
   - Key findings (3-5 bullet points)
   - Paths to visualizations created (in `scratchpad/plots/` format)
   - Confidence level in the analysis
   - Any caveats or limitations

## Testing Different Complexity Levels

- **Example 1 (Simple)**: Tests basic visualization capability
- **Example 2 (Medium)**: Tests statistical analysis and correlation calculations
- **Example 3 (Complex)**: Tests multi-faceted analysis, statistical testing, and comprehensive reporting

## Integration with Main Agent

In production, the main agent would:
1. Gather data using the `web-research-agent`
2. Save relevant data to `scratchpad/data/`
3. Delegate to `analysis-agent` with instructions + data path
4. Receive visualization paths and insights
5. Verify findings with `credibility-agent`
6. Compile everything into a PDF report using `generate_pdf_report`